#### Comments on training and validation set

The training set is created as described in the report. The validation set contains the same samples as the training set (without upsampling). The accuracy on the training set represents the training accuracy for relaxed categorical communication, while the accuracy on the validation set represents the training accuracy for true categorical communication. 

In [1]:
import pandas as pd

In [2]:
# grid search hyperparams

# {
#   "N": [20],
#   "n_epochs": [150],
#   "batch_size": [32],
#   "lr": [0.001],
#   "n_symbols": [200],
#   "receiver_embed_dim": [32, 64, 128],
#   "temperature": [1, 1.5, 2.0],
#   "temp_decay": [1, 0.995, 0.99],
#   "one_hot": [0, 1],
#   "n_layers": [1, 2, 3]
# }

In [3]:
# extract train/val loss and train/val accuracies from txt files

results = {"embedding_dim": [], 
           "temperature": [],
           "temp_decay": [],
           "n_layers": [],
           "one_hot": [],
           "n_layers": [],
           "loss_train": [], 
           "loss_val": [], 
           "aux_train": [], 
           "aux_val": []}

run_count = 0

for receiver_embed_dim in [32, 64, 128]: 
    for temperature in [1, 1.5, 2.0]:
        for temp_decay in [1., 0.995, 0.99]:
            for one_hot in [0, 1]:
                for n_layers in [1, 2, 3]:
                    
                    results["embedding_dim"].append(receiver_embed_dim)
                    results["temperature"].append(temperature)
                    results["temp_decay"].append(temp_decay)
                    results["one_hot"].append(bool(one_hot))
                    results["n_layers"].append(n_layers)
                    
                    result_file = open(str(run_count) + ".out", "rb")
                    lines = result_file.readlines()
                    result_file.close()
                    
                    train_results = str(lines[-2]).split(' ')
                    val_results = str(lines[-1]).split(' ')
                    train_loss = float(train_results[4].split(',')[0])
                    val_loss = float(val_results[4].split(',')[0])
                    train_aux = float(train_results[5].split('\\')[0].split('=')[1].split(",")[0])
                    val_aux = float(val_results[5].split('\\')[0].split('=')[1].split(",")[0])
                    
                    results["loss_train"].append(train_loss)
                    results["loss_val"].append(val_loss)
                    results["aux_train"].append(train_aux)
                    results["aux_val"].append(val_aux)
                
                    run_count +=1

In [21]:
# sort results by validation loss (lowest first)

# 20 runs with lowest validation losses (any number of layers)
results = pd.DataFrame(results)
sorted_results = results.sort_values("loss_val", ascending=True)
sorted_results[0:20]

,embedding_dim,temperature,temp_decay,n_layers,one_hot,loss_train,loss_val,aux_train,aux_val
101,64,2.0,0.995,3,True,0.025702,0.008391,0.994444,1.000000
161,128,2.0,0.990,3,True,0.011598,0.013624,0.996222,0.994444
77,64,1.5,1.000,3,True,0.035789,0.032634,0.990167,0.991667
131,128,1.5,1.000,3,True,0.021259,0.034424,0.990556,0.988889
155,128,2.0,0.995,3,True,0.039101,0.035293,0.992278,0.991667
137,128,1.5,0.995,3,True,0.031997,0.079049,0.987667,0.972222
47,32,2.0,0.995,3,True,0.069318,0.118501,0.980722,0.963889
107,64,2.0,0.990,3,True,0.043489,0.130458,0.988167,0.952778
149,128,2.0,1.000,3,True,0.000939,0.175667,0.999778,0.963889
89,64,1.5,0.990,3,True,0.099305,0.193023,0.947611,0.913889


In [20]:
# 20 runs with lowest validation losses (2 layers)

sorted_results = results.sort_values("loss_val", ascending=True)
sorted_results[sorted_results.n_layers==2][0:20]

,embedding_dim,temperature,temp_decay,n_layers,one_hot,loss_train,loss_val,aux_train,aux_val
154,128,2.0,0.995,2,True,0.042297,0.233596,0.987056,0.919444
106,64,2.0,0.990,2,True,0.075793,0.259407,0.970833,0.913889
160,128,2.0,0.990,2,True,0.060968,0.352334,0.985444,0.902778
52,32,2.0,0.990,2,True,0.156167,0.423883,0.944222,0.838889
100,64,2.0,0.995,2,True,0.090372,0.473014,0.968556,0.872222
46,32,2.0,0.995,2,True,0.130952,0.511842,0.962722,0.833333
88,64,1.5,0.990,2,True,0.154045,0.577730,0.936389,0.808333
28,32,1.5,0.995,2,True,0.197067,0.583845,0.925889,0.788889
142,128,1.5,0.990,2,True,0.179843,0.586163,0.925611,0.808333
136,128,1.5,0.995,2,True,0.152760,0.593408,0.941611,0.822222


In [16]:
# 20 runs with lowest validation losses (1 layer)

sorted_results = results.sort_values("loss_val", ascending=True)
sorted_results[sorted_results.n_layers==1][0:20]

,embedding_dim,temperature,temp_decay,n_layers,one_hot,loss_train,loss_val,aux_train,aux_val
51,32,2.0,0.990,1,True,0.650570,1.086643,0.815722,0.508333
105,64,2.0,0.990,1,True,0.608688,1.115510,0.834389,0.486111
81,64,1.5,0.995,1,True,0.615815,1.149885,0.833167,0.497222
27,32,1.5,0.995,1,True,0.759425,1.191204,0.784111,0.458333
159,128,2.0,0.990,1,True,0.587701,1.198249,0.848722,0.480556
45,32,2.0,0.995,1,True,0.393785,1.209917,0.907889,0.516667
33,32,1.5,0.990,1,True,0.933041,1.267445,0.684833,0.433333
135,128,1.5,0.995,1,True,0.612412,1.291061,0.836944,0.469444
87,64,1.5,0.990,1,True,0.857989,1.292715,0.719833,0.413889
141,128,1.5,0.990,1,True,0.740287,1.299688,0.781167,0.430556
